In [8]:
import tensorflow as tf

In [9]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [10]:
image_count = len(images_ds)
image_count

130

In [61]:
type(images_ds)

<tf.Tensor: shape=(), dtype=string, numpy=b'images\\dog\\been calculating dog years wrong....jpg'>

위의 함수를 통해 images_ds 변수에 모든 파일의 이름이 들어가 있는 상태이다. 위의 변수를 포함한 모든 tensorflow에 있는 Dataset형식의 클래스들은 [0]이런식으로 직접접근이 되지 않는 것 같다. 오로지 for문을 통해 접근이 가능한 것 같다. 아니면 list(dataset)을 하던가

In [56]:
for file in images_ds.take(3):
    print(file)
    print(file.numpy())

tf.Tensor(b'images\\dog\\How dogs contribute to your health and....jpg', shape=(), dtype=string)
b'images\\dog\\How dogs contribute to your health and....jpg'
tf.Tensor(b'images\\dog\\Best Hypoallergenic Dogs [Updated....jpg', shape=(), dtype=string)
b'images\\dog\\Best Hypoallergenic Dogs [Updated....jpg'
tf.Tensor(b'images\\dog\\List of dog breeds - Wikipedia.jpg', shape=(), dtype=string)
b'images\\dog\\List of dog breeds - Wikipedia.jpg'


위에서 .take(3)을하여 전체를 가져오는 대신 이미지 파일 이름을 3개 가져왔다. 가져왔을 때의 상태는 Tensor이고 numpy()를 통해 텐서 안에 있는 값 자체를 뽑아올 수 있다.
위처럼 tensorflow에서 Input을 관장하는 대부분의 클래스들은 .take() 메서드를 통해 전체를 가져오기 싫을 때 일부만 따로 가져올 수 있다.

In [66]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images\\dog\\The US Army is testing augmented....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\dog\\Soi Dog Foundation _ Ending The... (1).jpg'


셔플은 notion에서 설명한 대로 동작한다. 결과를 보면 위의 값과 다른 것을 볼 수 있다.

In [67]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [68]:
len(train_ds)
for i in train_ds:
    print(i)
    break

tf.Tensor(b'images\\dog\\been calculating dog years wrong....jpg', shape=(), dtype=string)


In [69]:
len(test_ds)

26

위에서 가져가야 할 점은 skip인데 take와 정반대로 take가 디스크에서 인자 개수만큼 가져오는 것이라면 skip은 인자개수만큼 건너뛴 뒤 그 다음부터 (아마 맨 끝까지?)가져오는 메서드이다.

In [70]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [71]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

현재까지 한 것은 이미지 파일 이름을 가져온 것 뿐이였다. 이제 할 일은 이미지 데이터 파일과 각 데이터에 대응하는 레이블을 가져오는 것이다. 위의 함수는 이미지 파일명 경로를 받으면 그 경로에서 label이름만을 따로 가져오는 함수이다. tf.strnigs.split을 사용한 이유는 file_path가 string이 아니라 텐서이기 때문에 텐서를 string처럼 다루기 위해 사용한 함수이다.

In [72]:
for file_path in train_ds.take(3):
    print(get_label(file_path))

tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(b'dog', shape=(), dtype=string)


위의 결과를 보면 성공적으로 가져온 것을 알 수 있다.

In [73]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [74]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

위의 함수는 파일 경로를 받으면 label과 이미지를 받아오는 함수이다. tf.io.read_file()함수는 파일경로에 있는 파일을 읽어오는 함수이고 입력은 텐서와 string을 받는 것 같다.  

In [75]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

위에서 map을 한 순간 process_image함수를 통해 각 원소들은 원래 파일명을 담고 있었지만 image파일값과 label을 가진 튜플로 변경이 되었다. map을 함과 동시에 디스크에서 이미지 파일을 불러온 것이다.

In [64]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****", label)

**** tf.Tensor(
[[[192.51562  171.51562   65.457886]
  [183.4336   167.4336    69.59766 ]
  [206.13672  195.13672  115.13672 ]
  ...
  [161.       150.        61.      ]
  [164.92902  155.92902   64.929016]
  [184.48438  170.48438   81.484375]]

 [[216.54688  199.01495  112.054016]
  [204.46484  191.46484   89.46484 ]
  [211.       199.       113.      ]
  ...
  [162.09375  151.09375   62.09375 ]
  [163.       153.        66.      ]
  [173.57422  164.57422   73.57422 ]]

 [[223.       209.       136.      ]
  [221.       207.       132.      ]
  [219.49908  207.49908  125.499084]
  ...
  [170.       160.        71.      ]
  [165.4961   155.4961    68.49609 ]
  [169.       162.        71.      ]]

 ...

 [[198.       201.        88.      ]
  [200.       204.        93.      ]
  [192.42188  195.42188   82.421875]
  ...
  [114.64935  151.64935   56.512634]
  [131.38727  167.38727   97.38727 ]
  [132.36719  164.42188   88.421875]]

 [[197.       202.        73.53192 ]
  [202.08203  207.082

take()함수를 통해 튜플 한개만 가져온 것이다. take를 할 때 디스크에서 정보를 가져오는 것이 아님에 유의하자

In [77]:
def scale(image, label):
    return image / 255, label

In [78]:
train_ds = train_ds.map(scale)

In [79]:
for i in train_ds.take(1):
    print(i)

(<tf.Tensor: shape=(128, 128, 3), dtype=float32, numpy=
array([[[0.7316167 , 0.74805355, 0.75981826],
        [0.72149634, 0.74403435, 0.7529024 ],
        [0.7931478 , 0.8205988 , 0.85981447],
        ...,
        [0.99189883, 0.99189883, 0.99189883],
        [0.979104  , 0.99479026, 0.9908687 ],
        [0.9432598 , 0.9628676 , 0.9863971 ]],

       [[0.735858  , 0.76436454, 0.78472733],
        [0.7302744 , 0.77341163, 0.7937548 ],
        [0.87793493, 0.8975428 , 0.9147772 ],
        ...,
        [0.9843137 , 0.99215686, 0.9882353 ],
        [0.9764706 , 0.99215686, 0.99607843],
        [0.93855554, 0.9660065 , 0.9895359 ]],

       [[0.7621855 , 0.7977247 , 0.80740607],
        [0.8171425 , 0.8556229 , 0.88148075],
        [0.9069393 , 0.9226256 , 0.9343903 ],
        ...,
        [0.96879596, 0.9844822 , 0.9884038 ],
        [0.96879596, 0.9844822 , 0.9884038 ],
        [0.93893754, 0.96271205, 0.98624146]],

       ...,

       [[0.9743872 , 0.95870095, 0.91164213],
        [0.9

위를 보면 i가 튜플인 것을 볼 수 있다. 